# Ejercicio con kNNs

Primero, descargamos un dataset sobre el que vamos a trabajar en nuestro ejercicio en un subdirectorio `data` de nuestro directorio de trabajo:

In [1]:
!curl -o data/diabetes.csv https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23873  100 23873    0     0  70008      0 --:--:-- --:--:-- --:--:-- 69804


A continuación, cargamos los módulos que utilizaremos en el ejercicio:

In [2]:
# Nuestros sospechosos habituales
import pandas as pd
import numpy as np

# También, como siempre, nos apoyamos en Scikit-Learn para hacer el split en training y test
from sklearn.model_selection import train_test_split
# Usaremos el preprocesador StandardScaler para no tener sesgos en los datos
# de entrada
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
# Importamos herramientas para evaluar el modelo. F1 es la media
# armónica de precision y recall
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

Ahora es el turno de cargar el dataset en un Pandas dataframe y de investigar un poco qué es lo que tenemos en términos de número de muestras y de features. Apoyaos en la documentación pública de Pandas para investigar cómo cargar un dataset (lo veremos en clase de todas formas). Escribid el código en los placeholders marcados con `#tu código aquí#`:

In [3]:
# Leemos el CSV que hemos descargado con la ayuda de los métos que nos ofrecen
# los dataframes de Pandas
dataset = pd.read_csv('data/diabetes.csv')
dataset.head()

# Vemos cuántas muestras tiene nuestro dataset
dataset.shape

# Miramos también qué pinta tiene el dataset desde el
# punto de vista de características y etiquetas, usando las capacidades de
# nuevo de los dataframes de Pandas
dataset.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [8]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
dataset.isnull()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
763,False,False,False,False,False,False,False,False,False
764,False,False,False,False,False,False,False,False,False
765,False,False,False,False,False,False,False,False,False
766,False,False,False,False,False,False,False,False,False


Trata de investigar si hay columnas del dataset que contienen valores que son nulos y que no tienen sentido. Como ejemplo, un grosor de piel de valor cero no tiene mucho sentido:

In [16]:
dataset.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [14]:
dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [20]:
# Escribe código para filtrar los valores nulos de 1 columna del DataFrame y visualizarlos,
# deberías poder hacerlo en 1 línea
dataset.Glucose.isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
763    False
764    False
765    False
766    False
767    False
Name: Glucose, Length: 768, dtype: bool

Ahora, para la columna correspondiente a la feature que has elegido explorar (por ejemplo, skinThickness), calcula la media de los valores:

In [22]:
# Calcula la media de los valores de 1 columna de tu DataFrame
dataset.SkinThickness.mean().round(2)

20.54

In [29]:
dataset2=dataset.SkinThickness.fillna(value=dataset.SkinThickness.mean())

In [30]:
dataset2.mean()

20.536458333333332

Lo que vamos a hacer ahora es sustituir los valores nulos por la media de todos los valores de la columna, con el objetivo de poder seguir contando con las muestras que los tienen. Como pista, debes hacer esa operación para todas estas columnas en el DataFrame: 'Glucose','BloodPressure','SkinThickness','BMI','Insulin'. Lo mejor es que escribas código que itere sobre esas columnas para realizar la operación:

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [51]:
# Almacena las columnas a alterar en una lista de Python
columns=dataset[['Glucose','BloodPressure','SkinThickness','BMI','Insulin']]

# Itera sobre cada elemento de la lista, calculando la media y posteriormente sustituyendo
#los ceros por el valor calculado de la media tu código aquí#
for i in columns:
    mean=columns.mean()
    columns.replace(0,mean)
columns

,Glucose,BloodPressure,SkinThickness,BMI,Insulin
0,148,72,35,33.6,0
1,85,66,29,26.6,0
2,183,64,0,23.3,0
3,89,66,23,28.1,94
4,137,40,35,43.1,168
...,...,...,...,...,...
763,101,76,48,32.9,180
764,122,70,27,36.8,0
765,121,72,23,26.2,112
766,126,60,0,30.1,0


In [50]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Antes de comenzar con el proceso de entrenamiento, como siempre, partimos el dataset en training y test. Elige un 20% del dataset para el conjunto de testing, dejando el 80% restante para el entrenamiento:

In [8]:
# Recuerda que la columna nueve (índice 8) es la que tiene
# nuestras etiquetas, y que el resto contiene las features.
X = dataset[dataset.columns[0:7]]
y = dataset[dataset.columns[8]]
# Haz el split, selecciona un 20% del dataset original como datos de test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8)

Lo siguiente será escalar los datos, ésta es una técnica de ingeniería de características que será necesaria para poder calcular distancias correctamente con KNNs (veremos más sobre esto en clase):

In [9]:
# Ahora escalamos los datos, de manera que todos los rangos van desde -1 hasta 1.
sc_X = StandardScaler()
# Hacemos training y transformación conjunta sobre el training set
X_train = sc_X.fit_transform(X_train)
# Tenemos que asegurarnos de que el testing set también está transformado
X_test = sc_X.transform(X_test)

Una buena estimación del número de vecinos suele ser la raíz cuadrada del número de características. Usa este número como tu configuración inicial de `n_neighbors` y procede crear y entrenar un kNN:

In [12]:
# Define el modelo, inicializando kNN con los datos seleccionados
cls = KNeighborsClassifier(n_neighbors=3)
# Entrena el modelo
cls.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

Finalmente, evalua el modelo sobre el dataset de test, y muestra la precisión obtenida:

In [13]:
print("Precisión sobre el juego de pruebas: {:.2f}".format(cls.score(X_test,y_test)))

Precisión sobre el juego de pruebas: 0.73


In [14]:
cls.score(X_train,y_train)

0.8371335504885994